# Reduced alphabet similarity

Since exact matches are cheap, one might wonder if there is a way to use a lossy encoding on the data and test for exact matches of an inexact encoding.  This is the way of restricted alphabets.

## Soundex

The soundex algorithm is a procedure for encoding family names developed by the U.S. Census bureau.   It is described in patents 1,261,167 and 1,435,663.  The Works Progress Administration used thousands of people to create indexes to the 1920, 1900, and 1880 U.S. Decennial censuses during the 1930s, copying some of the information onto cards that were sorted by Soundex number and transferred to microfilm.

Claire Prechtel-Kluskens
The WPA Census Soundexing Projects
Prologue Magazine. Spring 2002, Vol. 34, No. 1  
https://www.archives.gov/publications/prologue/2002/spring/soundex-projects.html


Soundex constructs a four-character symbol (one initial letter and three numbers) for each name.  This symbol represents only some of the essential letters and sounds in the name.   This dramatically reduces the number of (distinct) objects in the index, giving names with similar sounds (or misspellings) the same symbol.

The algorithm is as follows:
*  The first letter of the name is retained as the first letter of the code.
*  Double letters and letters with identical numeric codes are combined.
*  If two consonants are separated by a vowel, both consonants are encoded; if two consonants are separated by a H or W and have the same soundex number, they are combined.
*  The first three consonants in the name are replaced by numbers, excepting final S and Z; if there are fewer than three, add zeros.

ignore   A,E,I,O,U,Y
1        B,F,P,V
2        C,G,K,Q,X,S,Z
3        D,T
4        L
5        M,N
6        R

If we work through the algorithm, we can encode a handful of names:

CAMPBELL:  The first letter is C, 
*    second consonant is M which encodes as 5
*    the next consonants P and B both encode to the same code (1), so are combined.
*    the third consonant to encode is L (4)
* So CAMPBELL encodes as C514.

NICOLAE:  The first letter is N,
*    the second consonant is C which codes as 2,
*    the next consonant is L which codes as 4
*    and we have run out of consonants, so we code NICOLAE as N24 or N240.

Santhosh Thottingal has written an implementation of soundex that we can use since it's licensed under the GPL: https://github.com/libindic/soundex

In [4]:
!pip install soundex
from soundex import Soundex
soundex = Soundex().soundex
def soundex4(name):
    return soundex(name, length=4)[0:4]

And we can check that our answers above match Thottingal's implementation:

In [5]:
soundex4("CAMPBELL") , soundex4("NICOLAE") 

('C514', 'N24')

Let us calucate the Soundex codes for the names on the list of 162k most common surnames from the 2010 [US Decennial] census, desecribed in a technical report "FREQUENTLY OCCURRING SURNAMES IN THE 2010 CENSUS" by Joshua Comentz dated October 2016 
https://www.census.gov/topics/population/genealogy/data/2010_surnames.html



In [6]:
import pandas as pd
surnames2010 = pd.read_csv("Names_2010Census.csv")
surnames2010

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
1,JOHNSON,2,1932812,655.24,1483.42,58.97,34.63,0.54,0.94,2.56,2.36
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49
3,BROWN,4,1437026,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
4,JONES,5,1425470,483.24,3004.80,55.19,38.48,0.44,1,2.61,2.29
...,...,...,...,...,...,...,...,...,...,...,...
162249,DIETZMANN,160975,100,0.03,90062.93,96,0,0,(S),0,(S)
162250,DOKAS,160975,100,0.03,90062.96,94,(S),0,0,(S),(S)
162251,DONLEA,160975,100,0.03,90062.99,94,0,0,0,0,6
162252,DORIOTT,160975,100,0.03,90063.03,89,0,(S),0,5,(S)


Now that we have a list of 160k names, we can apply soundex to it.  Each of these names occurred more than 100 times in the 2010 US Decennial census, and together these names represent about 90% of the people enumerated.
Calculating the soundex codes for each name will group the names even more broadly; we will be able to see the collections of identically-coded names once we calculate them.

In [7]:
surnames2010.name.apply(soundex)

TypeError: 'float' object is not subscriptable

Aha, we've seen this before just a moment ago, somehow there is numeric data in the names column.  This time let us replace the column with a cleaned-up column:

In [9]:
surnames2010["CleanName"] = surnames2010.name.apply(str)

In [10]:
surnames2010.query("CleanName != name")

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic,CleanName
4909,NaN,4910,7170,2.43,60231.65,93.58,2.09,0.56,0.43,1.35,1.98,nan


In [ ]:
This suggests the same solution, turn off not-a-number identification:

In [11]:
surnames2010 = pd.read_csv("Names_2010Census.csv", na_filter=False )

In [ ]:
Try to `.apply` again:

In [13]:
codes = surnames2010.name.apply(soundex4)

Out of curiosity, what was the name that caused the parser to fail?

In [20]:
surnames2010.loc[4907:4911]

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic,soundex
4907,PRIDE,4908,7172,2.43,60226.79,41.06,52.84,0.31,0.45,3.12,2.22,P63
4908,ENCARNACION,4909,7171,2.43,60229.22,2.9,1.98,14.74,0.07,1.02,79.29,E526
4909,NULL,4910,7170,2.43,60231.65,93.58,2.09,0.56,0.43,1.35,1.98,N4
4910,STROTHER,4910,7170,2.43,60234.08,63.64,29.18,0.5,0.92,3.15,2.61,S363
4911,BINKLEY,4912,7159,2.43,60236.51,93.34,2.65,0.46,0.17,1.79,1.59,B524


In [21]:
# Put soundex codes into a new column
surnames2010["soundex"] = codes

In [22]:
surnames2010[["name", "soundex", "count"]]

,name,soundex,count
0,SMITH,S53,2442977
1,JOHNSON,J525,1932812
2,WILLIAMS,W452,1625252
3,BROWN,B65,1437026
4,JONES,J52,1425470
...,...,...,...
162249,DIETZMANN,D325,100
162250,DOKAS,D2,100
162251,DONLEA,D54,100
162252,DORIOTT,D63,100


It seems likely that all the names are distinct, but we can check:
    

In [23]:
len(surnames2010.name.value_counts()), len(surnames2010)

(162254, 162254)

Yes, they are all distinct. 

So, how many soundex codes are there for 162k names?

In [24]:
len(surnames2010.soundex.value_counts())

4162

There are only 4,000 codes.   This was the purpose of the indexing system; to put similarly-sounding names in the same file despite differences in spelling.  

In [25]:
surnames2010.soundex.value_counts()

M2      921
B2      818
B62     777
R2      768
L2      733
       ... 
U312      1
I436      1
Q341      1
T136      1
C464      1
Name: soundex, Length: 4162, dtype: int64

Note there are more than 900 names on this list with the code M2.  What are these names?

In [26]:
surnames2010.query("soundex=='M2'")

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic,soundex
277,MCCOY,278,110744,37.54,26593.57,66.88,26.71,0.46,0.89,2.43,2.64,M2
299,MEJIA,300,104057,35.28,27391.54,3.37,0.45,1.33,0.14,0.2,94.51,M2
422,MOSS,423,76908,26.07,31052.62,69.73,24.71,0.48,0.68,2.12,2.28,M2
438,MCGEE,439,74542,25.27,31462.07,62.25,31.53,0.45,0.73,2.72,2.32,M2
467,MACK,468,71056,24.09,32176.67,44.76,49.06,0.7,0.53,2.59,2.37,M2
...,...,...,...,...,...,...,...,...,...,...,...,...
161463,MIGOYA,160975,100,0.03,90036.28,15,0,0,0,0,85,M2
161467,MIHAS,160975,100,0.03,90036.42,91,0,(S),0,(S),(S),M2
161736,MAKAS,160975,100,0.03,90045.54,95,0,(S),0,(S),(S),M2
161785,MAZAIKA,160975,100,0.03,90047.20,100,0,0,0,0,0,M2


It now may be clear that while it is easy to calculate soundex from the name, the reverse procedure, determining the name from the soundex code, is not possible, barring truly unusual names that have unique encodings.

Here is the table of soundex codes and the number of names (out of our list of 162k):

In [27]:
surnames2010.soundex.value_counts()

M2      921
B2      818
B62     777
R2      768
L2      733
       ... 
U312      1
I436      1
Q341      1
T136      1
C464      1
Name: soundex, Length: 4162, dtype: int64

From this, we can examine a few of the seemingly-unique Soundex codes: 

In [33]:
(surnames2010.query("soundex=='Q523'")[["name", "soundex", "count"]], 
surnames2010.query("soundex=='I164'")[["name", "soundex", "count"]],
surnames2010.query("soundex=='P136'")[["name", "soundex", "count"]],
surnames2010.query("soundex=='C464'")[["name", "soundex", "count"]])

(            name soundex  count
 85498  QUANSTROM    Q523    219,
            name soundex  count
 85280  IBARROLA    I164    220,
              name soundex  count
 85205  PAPAPIETRO    P136    220,
              name soundex  count
 162125  COLARELLI    C464    100)

But we still can't count on these names the correct names that correspond to these soundex codes because our list of names was not comprehensve; the 2010 surnames dataset did not include 6 million distinct names that were reported  fewer than 100 times.

There are other phonetic encoding systems, including Double Metaphone and New York State Identification and Intelligence System Phonetic Code (NYIIS) that aim to identify similar-sounding names and words, and were designed with deficiencies in the Soundex encoding in mind.  

A phonetic name-matching system is used by the US Transportation Security Administration for enforcing the U.S. No-Fly List.  (Sara Kehaulani Goo. Faulty 'No-Fly' System Detailed.  Washington Post.  2004-10-09).  Matching on lossy phonetic encodings increases the number of travelers idenfied by the system as meriting closer scrutiny by requiring only similar-sounding names to names on the list. 

## Murphy alphabets

(Murphy 2000 10.1093/protein/13.3.149)

In biological sequence analysis, alignments between biological molecules inferred to have common ancestry are a central problem.  There are technologies that can turn information containing polymers into data representing the genetic information they contain.  